## Databases 

In [1]:
import os
import pandas as pd
import time

In [2]:
hostname = "127.0.0.1"

In [68]:
hospital = pd.read_json("../data/testData/hospital-sort.json").to_dict("records") 
hospitalAllPaths = pd.read_json("../data/testData/hospitall-allPaths.json").to_dict("list")

In [75]:
college = pd.read_json("../data/testData/college.json").to_dict("records")

### Mongo 

In [36]:
from pymongo import MongoClient

In [37]:
client = MongoClient(host=hostname,port=27017)

In [38]:
client

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)

In [39]:
local = client["local"]

In [59]:
allPaths = [dict(name=k, locations=v) for k,v in hospitalAllPaths.items()]

In [54]:
local["hospital"].insert_many(hospital)

In [60]:
local["hospitalPaths"].insert_many(allPaths)

In [77]:
local["college"].insert_many(college)

### Neo4j 

In [70]:
from py2neo import Graph, Node, Relationship

In [71]:
txpool = []
for e in hospital:
    start = Node("Hospital",name=e["start_name"],lat=e["start_lat"],lon=e["start_lon"])
    end = Node("Hospital",name=e["end_name"],lat=e["end_lat"],lon=e["end_lon"])
    
    r = Relationship(start,"NEAREST_PATH",end)
    txpool.append(r)
    
    for i in hospitalAllPaths.get(e["start_name"]): 
        #
        destination = Node("Hospital", name=i["end_name"], lat=e["end_lat"], lon=e["end_lon"]) 
        is_connected = Relationship(start, "IS_CONNECTED", destination, distance=i["distance"])
        txpool.append(is_connected)
    else: 
        continue 

In [72]:
unique = set([e["name"] for e in college])

for idx, collegeName in enumerate(unique):
    # Filter 
    pred = list(filter(lambda e: e["name"] == collegeName, college)) 
    getProgramName = list(map(lambda e: Node("ProgramName",name=e["programName"]), pred))
                              
    university = Node("University", name=pred[0]["name"], web=pred[0]["web"], city=pred[0]["city"])    
    
    [txpool.append(Relationship(university,"HAS_CAPACITY",e)) for e in getProgramName]

In [73]:
import time
graph = Graph(f"bolt://{hostname}:7687", auth=("neo4j", "12345678"))
tx = graph.begin()
start_time = time.time() 
[print(f"Processing {idx} out of {len(txpool)} - {tx.create(e)}") for idx, e in enumerate(txpool)]
print(f"Before commit: {time.time() - start_time}")
graph.commit(tx)
print(f"After commit: {time.time() - start_time}")

Processing 0 out of 13314 - None
Processing 1 out of 13314 - None
Processing 2 out of 13314 - None
Processing 3 out of 13314 - None
Processing 4 out of 13314 - None
Processing 5 out of 13314 - None
Processing 6 out of 13314 - None
Processing 7 out of 13314 - None
Processing 8 out of 13314 - None
Processing 9 out of 13314 - None
Processing 10 out of 13314 - None
Processing 11 out of 13314 - None
Processing 12 out of 13314 - None
Processing 13 out of 13314 - None
Processing 14 out of 13314 - None
Processing 15 out of 13314 - None
Processing 16 out of 13314 - None
Processing 17 out of 13314 - None
Processing 18 out of 13314 - None
Processing 19 out of 13314 - None
Processing 20 out of 13314 - None
Processing 21 out of 13314 - None
Processing 22 out of 13314 - None
Processing 23 out of 13314 - None
Processing 24 out of 13314 - None
Processing 25 out of 13314 - None
Processing 26 out of 13314 - None
Processing 27 out of 13314 - None
Processing 28 out of 13314 - None
Processing 29 out of 133

Processing 260 out of 13314 - None
Processing 261 out of 13314 - None
Processing 262 out of 13314 - None
Processing 263 out of 13314 - None
Processing 264 out of 13314 - None
Processing 265 out of 13314 - None
Processing 266 out of 13314 - None
Processing 267 out of 13314 - None
Processing 268 out of 13314 - None
Processing 269 out of 13314 - None
Processing 270 out of 13314 - None
Processing 271 out of 13314 - None
Processing 272 out of 13314 - None
Processing 273 out of 13314 - None
Processing 274 out of 13314 - None
Processing 275 out of 13314 - None
Processing 276 out of 13314 - None
Processing 277 out of 13314 - None
Processing 278 out of 13314 - None
Processing 279 out of 13314 - None
Processing 280 out of 13314 - None
Processing 281 out of 13314 - None
Processing 282 out of 13314 - None
Processing 283 out of 13314 - None
Processing 284 out of 13314 - None
Processing 285 out of 13314 - None
Processing 286 out of 13314 - None
Processing 287 out of 13314 - None
Processing 288 out o

Processing 498 out of 13314 - None
Processing 499 out of 13314 - None
Processing 500 out of 13314 - None
Processing 501 out of 13314 - None
Processing 502 out of 13314 - None
Processing 503 out of 13314 - None
Processing 504 out of 13314 - None
Processing 505 out of 13314 - None
Processing 506 out of 13314 - None
Processing 507 out of 13314 - None
Processing 508 out of 13314 - None
Processing 509 out of 13314 - None
Processing 510 out of 13314 - None
Processing 511 out of 13314 - None
Processing 512 out of 13314 - None
Processing 513 out of 13314 - None
Processing 514 out of 13314 - None
Processing 515 out of 13314 - None
Processing 516 out of 13314 - None
Processing 517 out of 13314 - None
Processing 518 out of 13314 - None
Processing 519 out of 13314 - None
Processing 520 out of 13314 - None
Processing 521 out of 13314 - None
Processing 522 out of 13314 - None
Processing 523 out of 13314 - None
Processing 524 out of 13314 - None
Processing 525 out of 13314 - None
Processing 526 out o

Processing 742 out of 13314 - None
Processing 743 out of 13314 - None
Processing 744 out of 13314 - None
Processing 745 out of 13314 - None
Processing 746 out of 13314 - None
Processing 747 out of 13314 - None
Processing 748 out of 13314 - None
Processing 749 out of 13314 - None
Processing 750 out of 13314 - None
Processing 751 out of 13314 - None
Processing 752 out of 13314 - None
Processing 753 out of 13314 - None
Processing 754 out of 13314 - None
Processing 755 out of 13314 - None
Processing 756 out of 13314 - None
Processing 757 out of 13314 - None
Processing 758 out of 13314 - None
Processing 759 out of 13314 - None
Processing 760 out of 13314 - None
Processing 761 out of 13314 - None
Processing 762 out of 13314 - None
Processing 763 out of 13314 - None
Processing 764 out of 13314 - None
Processing 765 out of 13314 - None
Processing 766 out of 13314 - None
Processing 767 out of 13314 - None
Processing 768 out of 13314 - None
Processing 769 out of 13314 - None
Processing 770 out o

Processing 978 out of 13314 - None
Processing 979 out of 13314 - None
Processing 980 out of 13314 - None
Processing 981 out of 13314 - None
Processing 982 out of 13314 - None
Processing 983 out of 13314 - None
Processing 984 out of 13314 - None
Processing 985 out of 13314 - None
Processing 986 out of 13314 - None
Processing 987 out of 13314 - None
Processing 988 out of 13314 - None
Processing 989 out of 13314 - None
Processing 990 out of 13314 - None
Processing 991 out of 13314 - None
Processing 992 out of 13314 - None
Processing 993 out of 13314 - None
Processing 994 out of 13314 - None
Processing 995 out of 13314 - None
Processing 996 out of 13314 - None
Processing 997 out of 13314 - None
Processing 998 out of 13314 - None
Processing 999 out of 13314 - None
Processing 1000 out of 13314 - None
Processing 1001 out of 13314 - None
Processing 1002 out of 13314 - None
Processing 1003 out of 13314 - None
Processing 1004 out of 13314 - None
Processing 1005 out of 13314 - None
Processing 100

...

Processing 12354 out of 13314 - None
Processing 12355 out of 13314 - None
Processing 12356 out of 13314 - None
Processing 12357 out of 13314 - None
Processing 12358 out of 13314 - None
Processing 12359 out of 13314 - None
Processing 12360 out of 13314 - None
Processing 12361 out of 13314 - None
Processing 12362 out of 13314 - None
Processing 12363 out of 13314 - None
Processing 12364 out of 13314 - None
Processing 12365 out of 13314 - None
Processing 12366 out of 13314 - None
Processing 12367 out of 13314 - None
Processing 12368 out of 13314 - None
Processing 12369 out of 13314 - None
Processing 12370 out of 13314 - None
Processing 12371 out of 13314 - None
Processing 12372 out of 13314 - None
Processing 12373 out of 13314 - None
Processing 12374 out of 13314 - None
Processing 12375 out of 13314 - None
Processing 12376 out of 13314 - None
Processing 12377 out of 13314 - None
Processing 12378 out of 13314 - None
Processing 12379 out of 13314 - None
Processing 12380 out of 13314 - None
P

Processing 12576 out of 13314 - None
Processing 12577 out of 13314 - None
Processing 12578 out of 13314 - None
Processing 12579 out of 13314 - None
Processing 12580 out of 13314 - None
Processing 12581 out of 13314 - None
Processing 12582 out of 13314 - None
Processing 12583 out of 13314 - None
Processing 12584 out of 13314 - None
Processing 12585 out of 13314 - None
Processing 12586 out of 13314 - None
Processing 12587 out of 13314 - None
Processing 12588 out of 13314 - None
Processing 12589 out of 13314 - None
Processing 12590 out of 13314 - None
Processing 12591 out of 13314 - None
Processing 12592 out of 13314 - None
Processing 12593 out of 13314 - None
Processing 12594 out of 13314 - None
Processing 12595 out of 13314 - None
Processing 12596 out of 13314 - None
Processing 12597 out of 13314 - None
Processing 12598 out of 13314 - None
Processing 12599 out of 13314 - None
Processing 12600 out of 13314 - None
Processing 12601 out of 13314 - None
Processing 12602 out of 13314 - None
P

Processing 12798 out of 13314 - None
Processing 12799 out of 13314 - None
Processing 12800 out of 13314 - None
Processing 12801 out of 13314 - None
Processing 12802 out of 13314 - None
Processing 12803 out of 13314 - None
Processing 12804 out of 13314 - None
Processing 12805 out of 13314 - None
Processing 12806 out of 13314 - None
Processing 12807 out of 13314 - None
Processing 12808 out of 13314 - None
Processing 12809 out of 13314 - None
Processing 12810 out of 13314 - None
Processing 12811 out of 13314 - None
Processing 12812 out of 13314 - None
Processing 12813 out of 13314 - None
Processing 12814 out of 13314 - None
Processing 12815 out of 13314 - None
Processing 12816 out of 13314 - None
Processing 12817 out of 13314 - None
Processing 12818 out of 13314 - None
Processing 12819 out of 13314 - None
Processing 12820 out of 13314 - None
Processing 12821 out of 13314 - None
Processing 12822 out of 13314 - None
Processing 12823 out of 13314 - None
Processing 12824 out of 13314 - None
P

Processing 13020 out of 13314 - None
Processing 13021 out of 13314 - None
Processing 13022 out of 13314 - None
Processing 13023 out of 13314 - None
Processing 13024 out of 13314 - None
Processing 13025 out of 13314 - None
Processing 13026 out of 13314 - None
Processing 13027 out of 13314 - None
Processing 13028 out of 13314 - None
Processing 13029 out of 13314 - None
Processing 13030 out of 13314 - None
Processing 13031 out of 13314 - None
Processing 13032 out of 13314 - None
Processing 13033 out of 13314 - None
Processing 13034 out of 13314 - None
Processing 13035 out of 13314 - None
Processing 13036 out of 13314 - None
Processing 13037 out of 13314 - None
Processing 13038 out of 13314 - None
Processing 13039 out of 13314 - None
Processing 13040 out of 13314 - None
Processing 13041 out of 13314 - None
Processing 13042 out of 13314 - None
Processing 13043 out of 13314 - None
Processing 13044 out of 13314 - None
Processing 13045 out of 13314 - None
Processing 13046 out of 13314 - None
P

Processing 13242 out of 13314 - None
Processing 13243 out of 13314 - None
Processing 13244 out of 13314 - None
Processing 13245 out of 13314 - None
Processing 13246 out of 13314 - None
Processing 13247 out of 13314 - None
Processing 13248 out of 13314 - None
Processing 13249 out of 13314 - None
Processing 13250 out of 13314 - None
Processing 13251 out of 13314 - None
Processing 13252 out of 13314 - None
Processing 13253 out of 13314 - None
Processing 13254 out of 13314 - None
Processing 13255 out of 13314 - None
Processing 13256 out of 13314 - None
Processing 13257 out of 13314 - None
Processing 13258 out of 13314 - None
Processing 13259 out of 13314 - None
Processing 13260 out of 13314 - None
Processing 13261 out of 13314 - None
Processing 13262 out of 13314 - None
Processing 13263 out of 13314 - None
Processing 13264 out of 13314 - None
Processing 13265 out of 13314 - None
Processing 13266 out of 13314 - None
Processing 13267 out of 13314 - None
Processing 13268 out of 13314 - None
P

In [74]:
graph.run("MATCH (n) RETURN COUNT(n)")

COUNT(n)
16206


In [12]:
#graph.run("MATCH (n) DETACH DELETE(n)")

(No data)